In [27]:
!pip install pdfminer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
import locale
# print(locale.getpreferredencoding())
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [30]:
!pip install docx2txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Import libaries
import regex as re
import pdfminer
from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import os
import docx2txt

In [ ]:
# Uploading the pdf resume and reading resume  
rootdir = "/content/drive/MyDrive/resume_parser/set3"
files = os.listdir(rootdir)
#print(files)
file_names = []
for file in os.listdir(rootdir):
    #print(os.path.join( file))
    file_names.append(os.path.join( "/content/drive/MyDrive/resume_parser/set3",file))
file_names
     

['/content/drive/MyDrive/resume_parser/set3/resume_1.pdf',
 '/content/drive/MyDrive/resume_parser/set3/resume_18.pdf',
 '/content/drive/MyDrive/resume_parser/set3/resume_10.pdf',
 '/content/drive/MyDrive/resume_parser/set3/resume_6.pdf',
 '/content/drive/MyDrive/resume_parser/set3/resume_4.pdf',
 '/content/drive/MyDrive/resume_parser/set3/resume_17.pdf',
 '/content/drive/MyDrive/resume_parser/set3/resume_3.pdf',
 '/content/drive/MyDrive/resume_parser/set3/resume_7.pdf',
 '/content/drive/MyDrive/resume_parser/set3/resume_5.pdf',
 '/content/drive/MyDrive/resume_parser/set3/resume_19.pdf',
 '/content/drive/MyDrive/resume_parser/set3/resume_2.pdf',
 '/content/drive/MyDrive/resume_parser/set3/resume_13.pdf',
 '/content/drive/MyDrive/resume_parser/set3/resume_14.pdf',
 '/content/drive/MyDrive/resume_parser/set3/resume_12.pdf',
 '/content/drive/MyDrive/resume_parser/set3/resume_8.pdf',
 '/content/drive/MyDrive/resume_parser/set3/resume_9.pdf',
 '/content/drive/MyDrive/resume_parser/set3/resum

In [ ]:
# Split all the resume line based on reqruirements
def open_pdf_file(file_name):
    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    pagenums = set()
    infile = open(file_name, 'rb')
    for page in PDFPage.get_pages(infile, pagenums):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close()

    result = []

    for line in text.split('\n'):
        line2 = line.strip()
        if line2 != '':
            result.append(line2)
    return (result)

In [ ]:
# Converting the resume to text 
def open_docx_file(file_name):
    temp = docx2txt.process(file_names[0])
    text = [line.replace('\t', ' ') for line in temp.split('\n') if line]
    text = [t for t in text if len(t) > 1]
    return (text)

In [ ]:
# Removing Punctuations from resume 
def remove_punctuations(line):
    return re.sub(r'(\.|\,)', '', line)

def preprocess_document(document):
    for index, line in enumerate(document):
        line = line.lower()
        line = remove_punctuations(line)
        
        line = line.split(' ')
        while '' in line:
            line.remove('')
            
        while ' '  in line:
            line.remove(' ')
            
            
        document[index] = ' '.join(line)
    return (document)
     

In [ ]:
# Extracting few more information from resume(phone_no)
def get_phone_no(document):
    #This function has to be further modified better and accurate results.
    #Possible phone number formats - Including +91 or just with the numbers.
    
    mob_num_regex = r'''(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|

                        [-\.\s]*\d{3}[-\.\s]??\d{4}|\d{5}[-\.\s]??\d{4})'''
    pattern = re.compile(mob_num_regex)
    matches = []
    for line in document:
        match = pattern.findall(line)
        for mat in match:
            if len(mat) > 9:
                matches.append(mat)

    return (matches)
     

In [35]:
def get_education(document):
    education_terms = []
    with open("/content/drive/MyDrive/resume_parser/education.txt", "r", encoding="utf-8") as file:
        education_terms = file.readlines()
    
    education_terms = [term.strip('\n') for term in education_terms]
    education = []
    for line in document:
        for word in line.split(' '):
            if len(word) > 2 and word in education_terms:
                if line not in education:
                    education.append(line)
    #print (education)
    return (education)
     

In [31]:
## Extracting Skill information from resume:
def get_skills(document):
    skill_terms = []
    
    with open("/content/drive/MyDrive/resume_parser/linkedin skill", 'r') as file:
        skill_terms = file.readlines()
    
    skill_terms = [term.strip('\n') for term in skill_terms]
    skills = []
    
    for line in document:
        words = line.split(' ')
        
        for word in words:
            if word in skill_terms:
                if word not in skills:
                    skills.append(word)
                    
        word_pairs = []
        for i in zip(words[:-1], words[1:]):
            word_pairs.append(i[0] + ' ' + i[1])   #This is to find skills like 'data science' i.e skills containint two words.    return (skills)
            
        for pair in word_pairs:
            if pair in skill_terms:
                if pair not in skills:
                    skills.append(pair)
                    
    return (skills)
     

In [ ]:
# Extracting experience information from resume:
def get_experience(document):
    pattern1 = re.compile(r'(jan(uary)?|feb(ruary)?|mar(ch)?|apr(il)?|may|jun(e)?|jul(y)?|aug(ust)?|sep(tember)?|oct(ober)?|nov(ember)?|dec(ember)?)(\s|\S)(\d{2,4}).*(jan(uary)?|feb(ruary)?|mar(ch)?|apr(il)?|may|jun(e)?|jul(y)?|aug(ust)?|sep(tember)?|oct(ober)?|nov(ember)?|dec(ember)?)(\s|\S)(\d{2,4})')
    pattern2 = re.compile(r'(\d{2}(.|..)\d{4}).{1,4}(\d{2}(.|..)\d{4})')
    pattern3 = re.compile(r'(\d{2}(.|..)\d{4}).{1,4}(present)')
    pattern4 = re.compile(r'(jan(uary)?|feb(ruary)?|mar(ch)?|apr(il)?|may|jun(e)?|jul(y)?|aug(ust)?|sep(tember)?|oct(ober)?|nov(ember)?|dec(ember)?)(\s|\S)(\d{2,4}).*(present)')
    patterns = [pattern1, pattern2, pattern3, pattern4]
    experience = []
    for index, line in enumerate(document):
        for pattern in patterns:
            exp = pattern.findall(line)
            if len(exp) > 0:
                experience.append(document[index:index+4])
                
    return (experience)
     

In [36]:
# Combining all the information from resume 
phone_nos = []
education_1 = []
education_2 = []
skills_1 = []
skills_2 = []
experience_1 = []
experience_2 = []

for file_name in file_names:
    print(file_name)
    if file_name.endswith('.pdf'):
        document = open_pdf_file(file_name)
    
    
    # email = get_email(document)
    phone_no = get_phone_no(document)
    document = preprocess_document(document)
    #print ('\n\n')
    #print (file_name)
    #print ('Email is {} phone number is {}'.format(email, phone_no))
        
    if len(phone_no) > 0:
        phone_nos.append(phone_no[0])
    else:
        phone_nos.append('')
    
    education = get_education(document)
    #print ('Education ', get_education(document))
    if len(education) > 1:
        education_1.append(education[0])
        education_2.append(education[1])
    elif len(education) == 1:
        education_1.append(education[0])
        education_2.append('')
    elif len(education) == 0:
        education_1.append('')
        education_2.append('')
        
    skills = get_skills(document)
    #print ('Skills ', skills)
    
    if len(skills) > 1:
        skills_1.append(skills[0])
        skills_2.append(skills[1])
    elif len(skills) == 1:
        skills_1.append(skills[0])
        skills_2.append('')
    elif len(skills) == 0:
        skills_1.append('')
        skills_2.append('')
        
    experience = get_experience(document)
    #print ('Experience ', get_experience(document))
    if len(experience) > 1:
        experience_1.append(experience[0])
        experience_2.append(experience[1])
    elif len(experience) == 1:
        experience_1.append(experience[0])
        experience_2.append('')
    elif len(experience) == 0:
        experience_1.append('')
        experience_2.append('')
     

/content/drive/MyDrive/resume_parser/set3/resume_1.pdf
/content/drive/MyDrive/resume_parser/set3/resume_18.pdf
/content/drive/MyDrive/resume_parser/set3/resume_10.pdf
/content/drive/MyDrive/resume_parser/set3/resume_6.pdf
/content/drive/MyDrive/resume_parser/set3/resume_4.pdf
/content/drive/MyDrive/resume_parser/set3/resume_17.pdf
/content/drive/MyDrive/resume_parser/set3/resume_3.pdf
/content/drive/MyDrive/resume_parser/set3/resume_7.pdf
/content/drive/MyDrive/resume_parser/set3/resume_5.pdf
/content/drive/MyDrive/resume_parser/set3/resume_19.pdf
/content/drive/MyDrive/resume_parser/set3/resume_2.pdf
/content/drive/MyDrive/resume_parser/set3/resume_13.pdf
/content/drive/MyDrive/resume_parser/set3/resume_14.pdf
/content/drive/MyDrive/resume_parser/set3/resume_12.pdf
/content/drive/MyDrive/resume_parser/set3/resume_8.pdf
/content/drive/MyDrive/resume_parser/set3/resume_9.pdf
/content/drive/MyDrive/resume_parser/set3/resume_16.pdf
/content/drive/MyDrive/resume_parser/set3/resume_11.pdf
/

In [37]:
# Making table by using pandas DataFrame
import pandas as pd
df = pd.DataFrame({'phone nos':phone_nos, 'education 1':education_1, 'education 2':education_2, 'skills 1':skills_1, 
              'skills 2':skills_2, 'experience 1':experience_1, 'experiece 2':experience_2})

In [38]:
df.to_excel('/content/drive/MyDrive/resume_parser/extracted_data.xlsx',index=False)

In [39]:

df

,phone nos,education 1,education 2,skills 1,skills 2,experience 1,experiece 2
0,,,,,,,
1,,,,,,,
2,9183447743,,,,,"[(cid:17) july2016–may2020, python&rprogrammin...",
3,6362090529,,,,,"[05/2018 – present, achievements/tasks, 636209...","[02/2017 – 08/2017, achievements/tasks, i work..."
4,,,,,,,
5,,btech computer science and engineering,,btech,engineering,[higher secondary education cbse 04/2015-04/20...,
6,73378 3406,of engineering after my high school graduation...,,engineering,,"[aug 16 - feb 17 research internship, research...",
7,,,,,,,
8,82108-3827,,,,,,
9,9972860492,,,,,"[sep 2017 - feb 2018, imaginxp, bachelor of mu...","[sep 2010 - aug 2013, wiztoonz academy of medi..."
